#Clasificación de Sentimientos con una Red Neuronal Multicapa (PyTorch)
##🎯 Objetivo
En esta actividad vas a construir una red neuronal feedforward multicapa (MLP) usando PyTorch. El objetivo es entrenarla para que pueda clasificar frases en español como positivas o negativas.

###Con esto vas a:

* Comprender cómo se arma una red con varias neuronas.

* Usar funciones de activación y entrenamiento automático.

* Observar cómo mejora respecto al perceptrón simple de la Actividad 1.

##🧰 1. Preparación del entorno
Importamos PyTorch y NumPy para comenzar.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

##🗂️ 2. Datos de entrenamiento
Usamos un conjunto de frases típicas de opiniones escritas en Argentina, etiquetadas como positivas (1) o negativas (0).

In [ ]:
frases = [
    "La verdad, este lugar está bárbaro. Muy recomendable",
    "Una porquería de servicio, nunca más vuelvo",
    "Me encantó la comida, aunque la música estaba muy fuerte",
    "El envío fue lento y el producto llegó dañado. Qué desastre",
    "Todo excelente. Atención de diez",
    "Qué estafa, me arrepiento de haber comprado",
    "Muy conforme con el resultado final",
    "No me gustó para nada la experiencia",
    "Superó mis expectativas, ¡gracias!",
    "No lo recomiendo, mala calidad"
]

etiquetas = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])  # 1 = Positivo, 0 = Negativo


##🧾 3. Construcción del vocabulario
Definimos manualmente un vocabulario con palabras que suelen aparecer en frases de opinión con carga positiva o negativa.

In [ ]:
vocabulario = [
    "bárbaro", "recomendable", "porquería", "nunca", "encantó",
    "fuerte", "desastre", "excelente", "estafa", "arrepiento",
    "conforme", "gustó", "superó", "gracias", "recomiendo", "mala"
]


##🧠 4. Preprocesamiento: vectorización de las frases
Cada frase se convierte en un vector binario (bag-of-words) que indica si contiene alguna de las palabras del vocabulario.

In [ ]:
def vectorizar(frase, vocabulario):
    tokens = frase.lower().split()
    return np.array([1 if palabra in tokens else 0 for palabra in vocabulario], dtype=np.float32)

X_np = np.array([vectorizar(frase, vocabulario) for frase in frases], dtype=np.float32)
y_np = etiquetas.astype(np.float32).reshape(-1, 1)

# Convertimos a tensores de PyTorch
X = torch.tensor(X_np)
y = torch.tensor(y_np)

##🧱 5. Definición del modelo (MLP)
Vamos a crear un modelo simple con una capa oculta, activación ReLU, y una salida sigmoide para predicción binaria.

In [ ]:
input_size = len(vocabulario)
hidden_size = 8

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

modelo = MLP()

##⚙️ 6. Entrenamiento del modelo
Definimos la función de pérdida y el optimizador. Entrenamos por varias épocas.

In [ ]:
criterio = nn.BCELoss()  # Binary Cross Entropy
optimizador = optim.Adam(modelo.parameters(), lr=0.01)

epocas = 200

for epoca in range(epocas):
    modelo.train()
    salida = modelo(X)
    loss = criterio(salida, y)

    optimizador.zero_grad()
    loss.backward()
    optimizador.step()

    if (epoca + 1) % 10 == 0:
        print(f"Época {epoca+1}, Pérdida: {loss.item():.4f}")


##🧪 7. Evaluación con frases nuevas
Probamos la red con frases que no estaban en el entrenamiento, para ver cómo generaliza.

In [ ]:
frases_prueba = [
    "No me gustó la atención, bastante mala",
    "Muy buena experiencia, todo excelente",
    "Una estafa total, no lo recomiendo",
    "Súper conforme con el servicio",
    "Nada que ver con lo prometido, una decepción"
]

# Vectorizamos las frases de prueba
X_prueba_np = np.array([vectorizar(frase, vocabulario) for frase in frases_prueba], dtype=np.float32)
X_prueba = torch.tensor(X_prueba_np)

# Predicción
modelo.eval()
with torch.no_grad():
    predicciones = modelo(X_prueba)

# Mostrar resultados
for frase, pred in zip(frases_prueba, predicciones):
    clase = "Positivo" if pred.item() >= 0.5 else "Negativo"
    print(f"Frase: '{frase}' => Sentimiento predicho: {clase} ({pred.item():.2f})")

##💬 Reflexión final
###👉 ¿Qué aprendimos?

* Cómo implementar y entrenar una red neuronal multicapa para análisis de sentimiento.

* Cómo preprocesar texto en español usando bag-of-words.

* Las ventajas del MLP frente al perceptrón simple.

* Limitaciones: aún no capta el orden de las palabras ni el contexto secuencial.

➡️ En la próxima actividad aprenderemos a usar redes recurrentes (LSTM) para incorporar secuencia y memoria en el procesamiento de texto. ¡Nos acercamos a modelos más cercanos al lenguaje humano!